In [2]:
from googleapiclient import discovery
import httplib2
from oauth2client.client import GoogleCredentials

In [3]:
DISCOVERY_URL='https://{api}.googleapis.com/$discovery/rest?version={apiVersion}'


def get_vision_service():
    credentials = GoogleCredentials.get_application_default()
    return discovery.build('vision', 'v1', credentials=credentials,
                           discoveryServiceUrl=DISCOVERY_URL)

In [25]:
import base64

def detect_face(face_file, max_results=10):
    """Uses the Vision API to detect faces in the given file.

    Args:
        face_file: A file-like object containing an image with faces.

    Returns:
        An array of dicts with information about the faces in the picture.
    """
    image_content = face_file.read()
    batch_request = [{
        'image': {
            'content': base64.b64encode(image_content).decode('UTF-8')
            },
        'features': [{
            'type': 'FACE_DETECTION',
            'maxResults': max_results,
            }]
        }]

    service = get_vision_service()
    request = service.images().annotate(body={
        'requests': batch_request,
        })
    response = request.execute()

    return response['responses'][0]['faceAnnotations']

In [6]:
from PIL import Image, ImageDraw

def highlight_faces(image, faces, output_filename):
    """Draws a polygon around the faces, then saves to output_filename.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
      output_filename: the name of the image file to be created, where the faces
          have polygons drawn around them.
    """
    im = Image.open(image)
    draw = ImageDraw.Draw(im)

    for face in faces:
        box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in face['fdBoundingPoly']['vertices']]
        draw.line(box + [box[0]], width=5, fill='#00ff00')

    del draw
    im.save(output_filename)

In [6]:
from PIL import Image, ImageDraw
import numpy as np

def extract_faces(image, faces):
    """Extract faces from image.

    Args:
      image: a file containing the image with the faces.
      faces: a list of faces found in the file. This should be in the format
          returned by the Vision API.
    Return array of faces
    """
    arr = np.zeros()
    I = Image.open(image)

    for face in faces:
        box = [(v.get('x', 0.0), v.get('y', 0.0)) for v in face['fdBoundingPoly']['vertices']]
        Arr.line(box + [box[0]], width=5, fill='#00ff00')
    
    return arr

In [26]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/oleksii/Downloads/Google Cloud Vision-9785cb814b4e.json"

def main(input_filename, output_filename):
    with open(input_filename, 'rb') as image:
        faces = detect_face(image)
        print('Found %s face%s' % (len(faces), '' if len(faces) == 1 else 's'))

        print('Writing to file %s' % output_filename)
        # Reset the file pointer, so we can read the file again
        image.seek(0)
        highlight_faces(image, faces, output_filename)

Found 7 faces
Writing to file /home/oleksii/Downloads/out.jpg


In [42]:
from glob import iglob
import os

lst = iglob("/media/sf_host_dev/github/face_identification/test/*.jpg")
for l in lst:
    in_dir = os.path.dirname(l)
    out_dir = "{}_out".format(in_dir)
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    file_name = os.path.basename(l)
    out_file_name = "{}/{}".format(out_dir, file_name)
    main(l, out_file_name)

Found 4 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/g1.jpg
Found 6 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/g3.jpg
Found 10 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/g4.jpg
Found 7 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/g7.jpg
Found 1 face
Writing to file /media/sf_host_dev/github/face_identification/test_out/s1.jpg
Found 1 face
Writing to file /media/sf_host_dev/github/face_identification/test_out/s3.jpg
Found 4 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/s4.jpg
Found 2 faces
Writing to file /media/sf_host_dev/github/face_identification/test_out/s6.jpg
Found 1 face
Writing to file /media/sf_host_dev/github/face_identification/test_out/s7.jpg


In [49]:
file = "/media/sf_host_dev/github/face_identification/test/s7.jpg"
with open(file, "rb") as face_file:
    image_content = face_file.read()
    batch_request = [{
        'image': {
            'content': base64.b64encode(image_content).decode('UTF-8')
            },
        'features': [{
            'type': 'FACE_DETECTION',
            'maxResults': 10,
            }]
        }]

    service = get_vision_service()
    request = service.images().annotate(body={
        'requests': batch_request,
        })
    response = request.execute()


In [50]:
import json

json.dumps(response)

'{"responses": [{"faceAnnotations": [{"joyLikelihood": "LIKELY", "rollAngle": -1.4918807, "underExposedLikelihood": "VERY_UNLIKELY", "tiltAngle": -4.88615, "angerLikelihood": "VERY_UNLIKELY", "panAngle": 9.5356731, "fdBoundingPoly": {"vertices": [{"y": 1178, "x": 2525}, {"y": 1178, "x": 3571}, {"y": 2224, "x": 3571}, {"y": 2224, "x": 2525}]}, "detectionConfidence": 0.99962556, "sorrowLikelihood": "VERY_UNLIKELY", "landmarks": [{"position": {"y": 1513.1189, "x": 2859.5859, "z": -0.0058395704}, "type": "LEFT_EYE"}, {"position": {"y": 1506.6356, "x": 3272.2468, "z": 70.254463}, "type": "RIGHT_EYE"}, {"position": {"y": 1442.5811, "x": 2694.2473, "z": 11.156118}, "type": "LEFT_OF_LEFT_EYEBROW"}, {"position": {"y": 1432.7175, "x": 2985.8958, "z": -73.944435}, "type": "RIGHT_OF_LEFT_EYEBROW"}, {"position": {"y": 1423.5071, "x": 3187.6152, "z": -39.011433}, "type": "LEFT_OF_RIGHT_EYEBROW"}, {"position": {"y": 1403.416, "x": 3425.27, "z": 128.80548}, "type": "RIGHT_OF_RIGHT_EYEBROW"}, {"positio

In [ ]:
G_file = "/media/sf_host_dev/github/face_identification/test/g7.jpg"
S_file = "/media/sf_host_dev/github/face_identification/test/s7.jpg"

